In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
%load_ext nb_black
%load_ext nb_mypy

In [ ]:
import nbimporter
from Exercise03AI import Exercise03AI

# Aufgabe 04: Minimax (Simplified Evaluation Function) mit Memoisierung

Dieses Notebook implementiert die Memoisierungsfunktion für den Minimax-Algorithmus zur Berechnung des nächsten Zuges im Mittelspiel. Hierbei wird die `minimax`-Funktion der Super-Klasse (`Exercise03AI`) aufgerufen und durch die nun implementierte Memoisierung beschleunigt.

In [ ]:
import chess_custom as chess


class Exercise04AI(Exercise03AI):
    """Chooses middle game moves using minimax algorithm and piece square tables"""

    def __init__(self, **kwargs) -> None:
        super().__init__(**kwargs)
        self.cache: dict[tuple, Any] = {}
        self.cache_hits = 0

    def reset(self) -> None:
        """Resets all internal variables"""
        super().reset()
        self.cache = {}
        self.cache_hits = 0

## Memoisierungsfunktion

Die Klasse wird um eine Variable `cache` erweitert. Initial ist `cache` ein leeres Dictionary, welches als Schlüssel ein Tupel annimmt und als Wert ein beliebiges Objekt zurückgibt.

Die Memoisierungsfunktion `memoize_minimax` nimmt die Minimax-Funktion `minimax` als Argument und gibt eine memoisierte Version `minimax_memoized` dieser Funktion zurück.

Die memoisierte Funktion `minimax_memoized` versucht zunächst, den Rückgabewert der Ursprungsfunktion aus dem `cache` auszulesen. Dabei werden als Schlüssel die  Funktionsargumente zusammen mit weiteren Attributen verwendet. Beinhaltet der `cache` ein Ergebnis für die gegebenen Argumente `args`, so wird dieses zurückgegeben. Andernfalls wird die Ursprungsfunktion `f` aufgerufen, der Rückgabewert wird für die verwendeten Argumente im `cache` hinterlegt und schlussendlich wird der berechnete Wert zurückgegeben.

### Cache-Schlüssel

Eine wichtige Rolle spielt der Key, welcher für die Speicherung bereits berechneter Ergebnisse im `cache`-Dictionary verwendet wird. Um den Cache möglichst effektiv nutzen zu können, sollten nur die Attribute in die Berechnung des Schlüssels miteinbezogen werden, welche auch durch den Minimax-Algorithmus verwendet werden und sich im Laufe des Spieles verändern. Die Funktion `get_key` berechnet den Cache-Schlüssel und gibt ein Tupel mit den folgenden Elementen zurück:
- Ein sog. Transpositions-Schlüssel, d. h. eine Repräsentation des aktuellen Boards. Dieser Schlüssel enthält jeweils eine Bitmap pro Farbe (definiert Farbe und Position der Figuren), eine Bitmap pro Figurentyp, einen boolean für den aktuellen Spieler, eine Bitmap für die Rochaderechte und eine Bitmap für die möglichen En-passant-Züge.
- Die Anzahl der bisher bekannten Transpositionen zu dieser Stellung, d. h. wie oft sich die aktuelle Stellung bereits im Spielverlauf wiederholt hat. Zwei Stellungen gelten dann als gleich, wenn ihre Transpositions-Schlüssel dieselben sind. Dieses Attribut wird benötigt um ein unbeabsichtigtes Remis durch fünfmalige Zugwiederholung (`fivefold-repetition`) zu verhindern.
- Der Zähler der Halbzüge seit dem letzten Bauernzug oder Schlagen einer Figur. Dieses Attribut wird benötigt um ein unabsichtliches Remis durch die 50-Züge-Regel (`fifty-move`) zu verhindern. Aus Effizienzgründen wird der Zähler nur dann inkludiert wenn er für die Berechnung relevant ist, d. h. ein solches Remis mit der aktuellen Tiefe auftreten kann. Ansonsten wird der statische Wert `-42` verwendet.
- Die aktuelle Suchtiefe der Evaluierung.

*Hinweis:* Das Argument `last_eval` der Minimax-Funktion wird nicht als Cache-Schlüssel verwendet, da es direkt vom Transpositions-Schlüssel abhängt.

Die Funktion `memoize_minimax` wird als _Decorator_ auf die `minimax` Funktion angewendet.

### Seiteneffekte

In Ausnahmefällen kann die hier genutze Memoisierung von Minimax zu einer anderen Evaluierung und damit zu einer anderen Zugentscheidung führen. Dies ist dem Umstand geschuldet, dass ein Ergebnis aus dem Cache mit einer älteren Transpositionstabelle berechnet wurde und eine der ausgewerteten Stellungen nun mittlerweile möglicherweise als Remis gewertet werden müsste. Im schlimmsten Fall kann dieser Umstand zu einem unbeabsichtigten Remis führen, im besten Fall führt er nur zu einer leicht anderen Zugfolge mit demselben Spielausgang. Dieses Problem tritt allerdings sehr selten auf: Innerhalb von 540 Test-Spielen gegen andere AI-Versionen konnte nur bei einem Spiel eine andere Zugreihenfolge (mit demselben Spielergebnis) beobachtet werden. Die meisten Transpositionen treten in kurzer Folge (oft mit einer Länge von 4 Halbzügen) auf. Diese werden dann durch den veränderten Cache-Schlüssel zuverlässig erkannt, wordurch ein unabsichtliches Remis durch eine fünfmalige Zugwiederholung verhindert wird.
Eine allgemeine Lösung für diese Problem besteht darin, die gesamte Transpositionstabelle als Bestandteil des Cache-Schlüssels zu verwenden. Dies führt aber dazu, dass der Cache dann nur sehr selten verwendet werden kann und die Geschwindigkeitsvorteile der Memoisierung nahezu vollständig verloren gehen. Diese Probleme bei der Memoisierung werden u. a. in dem letzten Absatz [dieses Wikipedia-Abschnitts](https://en.wikipedia.org/wiki/Transposition_table#Functionality) treffend zusammengefasst:
> "[The] Use of a transposition table can lead to incorrect results if the graph-history interaction problem is not studiously avoided. This problem arises in certain games because the history of a position may be important. [...] Another examplen [in chess] is draw by repetition: given a position, it may not be possible to determine whether it has already occurred. A solution to the general problem is to store history information in each node of the transposition table, but this is inefficient and rarely done in practice."

Eine weitere Diskussion des Problems findet sich in [diesem Stack Overflow-Beitrag](https://stackoverflow.com/questions/69372792/chess-programming-minimax-detecting-repeats-transposition-tables).

In [ ]:
from typing import Any


class Exercise04AI(Exercise04AI):  # type: ignore
    @staticmethod
    def get_key(board: chess.Board, depth: int) -> tuple:
        """Calculates a key that uniquely identifies a given board"""
        return (
            (key := board._transposition_key()),
            board._transposition_table[key],
            board.halfmove_clock if board.halfmove_clock >= (50 - depth) else -42,
            depth,
        )

    @staticmethod
    def memoize_minimax(minimax):
        def minimax_memoized(
            self, board: chess.Board, depth: int, current_evaluation: int
        ):
            key = Exercise04AI.get_key(board, depth)
            if key in self.cache:
                self.cache_hits += 1
                return self.cache[key]
            result = minimax(self, board, depth, current_evaluation)
            self.cache[key] = result
            return result

        return minimax_memoized

    @memoize_minimax
    def minimax(self, *args) -> int:
        """Memoized version of the Exercise03AI minimax implementation"""
        return super().minimax(*args)

## Debugging Bereich

Die folgenden Zellen enthalten Unit-Tests der oben implementierten Funktionen.

In [ ]:
from AIBaseClass import ChessAI
import Exercise03AI

In [ ]:
# Create player and board
unit_test_player = Exercise04AI(player_name="Ex04AI", search_depth=2)
board = chess.Board("5rk1/1b3p2/8/3p4/3p2P1/2Q4B/5P1K/R3R3 b - - 0 36")
FULL_EVAL = 1240
board

In [ ]:
# Test key generation
def test_key_generation(unit_test_player: ChessAI, board: chess.Board):
    key = Exercise04AI.get_key(board, depth=42)
    (transposition_key, transpositions, half_moves_key, depth) = key
    assert transpositions == 0, "Transpositions does not match expected value!"
    assert half_moves_key == -42, "Half moves key does not match expected value!"
    assert depth == 42, "Depth does not match expected value!"
    print(f"Transposition Key: {transposition_key}")
    print(f"Transpositions: {transpositions}")
    print(f"Half moves key: {half_moves_key}")
    print(f"Depth: {depth}")

In [ ]:
# Test minimax
def test_minimax(unit_test_player: ChessAI, board: chess.Board):
    unit_test_player.cache = {}  # Clear cache
    Exercise03AI.test_minimax(unit_test_player, board)
    assert unit_test_player.cache != {}, "Cache is empty!"
    print(f"Elements in cache: {len(unit_test_player.cache)}")

In [ ]:
test_minimax(unit_test_player, board)

In [ ]:
# Test minimax again (now memoized)
def test_memoized_minimax(unit_test_player: ChessAI, board: chess.Board):
    unit_test_player.cache_hits = 0  # Reset cache hits counter
    mm_evaluation, mm_move = unit_test_player.minimax(board, 2, FULL_EVAL)
    assert mm_evaluation == 355, "Minimax evaluation does not match expected value!"
    assert mm_move.uci() == "d4c3", "Minimax move does not match expected value!"
    assert unit_test_player.cache_hits == 1, "Cache was not used correctly!"
    print(f"Minimax Evaluation: {mm_evaluation}")
    print(f"Minimax Move: {mm_move}")
    print(f"Cache hits: {unit_test_player.cache_hits}")

In [ ]:
# Test next move function (with memoized minimax result)
Exercise03AI.test_next_move(unit_test_player, board)

In [ ]:
# Test reset function
def test_reset(unit_test_player: ChessAI, _: chess.Board):
    Exercise03AI.test_reset(unit_test_player, board)
    assert unit_test_player.cache == {}, "Reset was not successful! (cache)"
    assert unit_test_player.cache_hits == 0, "Reset was not successful! (cache_hits)"

In [ ]:
test_reset(unit_test_player, board)

## Temporärer Bereich

Der folgende Bereich dient zum temporären Debuggen und kann nicht-funktionierenden Code enthalten. Dieser Bereich wird vor der Abgabe entfernt.